In [2]:
print('hello world') 

hello world


## 首先，先将sqlite表导入数据库

In [13]:
import json

# 输入和输出文件路径
input_file = r"D:\PythonProject\livesqlbench\livesqlbench-main\dev.json"
output_file = r"D:\PythonProject\livesqlbench\livesqlbench-main\world_1.json"

# 加载整个 JSON 文件
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)  # 此处是 load 而不是 loads/逐行读取


filtered_data = [item for item in data if item.get("db_id") == "world_1"]

# 保存为 JSON（数组格式）
with open(output_file, "w", encoding="utf-8") as f_out:
    json.dump(filtered_data, f_out, ensure_ascii=False, indent=2)

print(f"✅ 共提取 {len(filtered_data)} 条 db_id='world_1' 的记录，结果保存在：{output_file}")


✅ 共提取 52 条 db_id='world_1' 的记录，结果保存在：D:\PythonProject\livesqlbench\livesqlbench-main\world_1.json


In [4]:
import json

input_file = r"D:\PythonProject\livesqlbench\livesqlbench-main\bike_1_filtered.json"
output_file = r"D:\PythonProject\livesqlbench\livesqlbench-main\bike_1_few_shot_examples.json"

# 加载原始数据
with open(input_file, "r", encoding="utf-8") as f:
    data = json.load(f)

# 仅提取 question 和 query
few_shot_examples = [{"input": item["question"], "query": item["query"]} for item in data]

# 保存 Few-Shot 格式
with open(output_file, "w", encoding="utf-8") as f_out:
    json.dump(few_shot_examples, f_out, ensure_ascii=False, indent=2)

print(f"✅ 已生成 {len(few_shot_examples)} 个 Few-Shot 示例，保存在：{output_file}")


✅ 已生成 54 个 Few-Shot 示例，保存在：D:\PythonProject\livesqlbench\livesqlbench-main\bike_1_few_shot_examples.json


In [1]:
from langchain_community.utilities.sql_database import SQLDatabase

# 假设你有一个 bike_1.sqlite 文件
sqlite_path = r"D:\PythonProject\livesqlbench\livesqlbench-main\database\formula_1\formula_1.sqlite"

# 构造 SQLite 的连接 URI
sqlite_uri = f"sqlite:///{sqlite_path}"

# 创建 SQLDatabase 实例
db = SQLDatabase.from_uri(sqlite_uri)

# 测试输出
print("数据库方言：", db.dialect)
print("可用表名：", db.get_usable_table_names())
print("表结构信息：", db.table_info)


数据库方言： sqlite
可用表名： ['circuits', 'constructorResults', 'constructorStandings', 'constructors', 'driverStandings', 'drivers', 'lapTimes', 'pitStops', 'qualifying', 'races', 'results', 'seasons', 'status']
表结构信息： 
CREATE TABLE "constructorResults" (
	"constructorResultsId" INTEGER, 
	"raceId" INTEGER, 
	"constructorId" INTEGER, 
	points REAL, 
	status TEXT, 
	PRIMARY KEY ("constructorResultsId"), 
	FOREIGN KEY("constructorId") REFERENCES constructors ("constructorId"), 
	FOREIGN KEY("raceId") REFERENCES races ("raceId")
)

/*
3 rows from constructorResults table:
constructorResultsId	raceId	constructorId	points	status
1	18	1	14.0	NULL
2	18	2	8.0	NULL
3	18	3	9.0	NULL
*/


CREATE TABLE "constructorStandings" (
	"constructorStandingsId" INTEGER, 
	"raceId" INTEGER, 
	"constructorId" INTEGER, 
	points REAL, 
	position INTEGER, 
	"positionText" TEXT, 
	wins INTEGER, 
	PRIMARY KEY ("constructorStandingsId"), 
	FOREIGN KEY("constructorId") REFERENCES constructors ("constructorId"), 
	FOREIGN KE

In [2]:

table_info = db.table_info  # 这是一个字符串

# 写入到文本文件
with open("table_info.txt", "w", encoding="utf-8") as f:
    f.write(table_info)


## 生成一个table description，目的是可以让agent动态选表，把最精准的表传给模型进行nl2sql生成

-----------------------------------------------------------------------------------------------------------

In [ ]:
import os
os.environ['LANGSMITH_TRACING'] = 'true'
os.environ['LANGSMITH_API_KEY']  = 'N'
os.environ['LANGSMITH_PROJECT'] = 'livsqlbench'

#### 原始未经过rag+提示词工程的结果

In [ ]:
from langchain.chains import create_sql_query_chain
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="deepseek-chat", temperature=0,base_url="https://api.gpt.ge/v1",
            api_key="API KEY")  
generate_query = create_sql_query_chain(llm, db=db)
query = generate_query.invoke({"question": "如果库尔勒市所在的国家国民生产总值为1982268，给出人均国民生产总值最高的国家所在的大洲的名字"})
print(query)

To answer this question, we need to:
1. First find the country where the city "库尔勒市" (Korla) is located
2. Verify that this country's GNP is 1982268
3. Then find the country with the highest GNP per capita (GNP / Population)
4. Finally, return the continent of that country

Here's the SQL query:

```sql
WITH KorlaCountry AS (
    SELECT c."Code", c."Name", c."Continent", c."GNP", c."Population"
    FROM city ct
    JOIN country c ON ct."CountryCode" = c."Code"
    WHERE ct."Name" = '库尔勒市'
),
HighGNPPC AS (
    SELECT "Code", "Name", "Continent", "GNP"/"Population" AS GNPPC
    FROM country
    WHERE ("Code" = (SELECT "Code" FROM KorlaCountry WHERE "GNP" = 1982268))
    OR (1 = 1)
    ORDER BY GNPPC DESC
    LIMIT 1
)
SELECT "Continent" 
FROM HighGNPPC;
```

However, I notice there might be an issue with the query. The city "库尔勒市" (Korla) doesn't appear to be in the sample data provided, so the first part of the query might not return any results. Let me modify the approach to focus on 

In [6]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

C:\Users\felix\AppData\Local\Temp\ipykernel_31384\631663928.py:2: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  execute_query = QuerySQLDataBaseTool(db=db)


'Error: (sqlite3.OperationalError) near "I": syntax error\n[SQL: I notice that the question asks about the song "Gentleman", but looking at the sample data from the singer table, I don\'t see this song listed in any of the sample rows. However, I\'ll create a query to find singers who have "Gentleman" as their song and show their name and age at the time of the song\'s release.\n\nSQLQuery: \nSELECT "Name", "Age" - (date(\'now\') - "Song_release_year") AS "Age_at_release" \nFROM singer \nWHERE "Song_Name" = \'Gentleman\' \nLIMIT 5;\n\nNote: There appears to be a potential typo in the query - the song name is written as \'Gentleman\' instead of \'Gentleman\'. However, since we don\'t see this song in the sample data, the query might return no results. Also, the age calculation might need adjustment depending on how the data is stored.\n\nWould you like me to modify the query in any way? Perhaps to look for similar song names or to handle the age calculation differently?]\n(Background on

In [7]:
chain = generate_query | execute_query # 组合链,先生成查询语句，再执行查询
result = chain.invoke({"question": "2015年6月2日12:47:02，圣何塞文娱中心站可用的自行车停靠点有多少"})
print(result)

[(3,)]


In [25]:
# 查看default prompt
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

## 什么是 Few-Shot 学习？

这种技术指的是给模型提供少量精选的自然语言 → SQL 示例，帮助它更好地理解和生成精准的 SQL 查询。Few-shot 学习能够显著提高模型从自然语言中准确理解用户意图并生成正确 SQL 的能力，桥接“人类语言”与“数据库查询”之间的差距。

### 在 LangChain 中整合 Few-Shot 示例

第一步是挑选一组覆盖常见查询类型和复杂度的例子。最好能涵盖用户最可能提出的问题类型。

In [27]:
examples =[
  {
    "input": "2015年6月2日12:47:02，圣何塞文娱中心站不可用的自行车停靠点有多少？",
    "query": "SELECT A.dock_count - B.docks_available AS n_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE B.time = \"2015-06-02 12:47:02\" AND A.name = \"San Jose Civic Center\""
  },
  {
    "input": "如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？",
    "query": "SELECT 20 - B.docks_available AS n_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE B.time = \"2015-06-02 12:47:02\" AND A.name = \"San Jose Civic Center\""
  },
  {
    "input": "金门之城总共有多少个自行车站，比帕洛阿托少多少？",
    "query": "SELECT ( SELECT COUNT ( * ) FROM station WHERE city = \"Palo Alto\" ) - COUNT ( * ) AS diff FROM station WHERE city = \"San Francisco\""
  },
  {
    "input": "金门之城总共有多少个自行车站？如果帕洛阿托的自行车站数量是它自己的两倍还多10个，那金门之城的自行车站数量比帕洛阿托少多少？",
    "query": "SELECT ( SELECT COUNT ( * ) * 2 + 10 FROM station WHERE city = \"Palo Alto\" ) - COUNT ( * ) AS diff FROM station WHERE city = \"San Francisco\""
  },
  {
    "input": "在2015年6月2日12:46:02这个时刻，那些建于2013年8月20日的车站的可用自行车停靠点一共有多少，不可用的自行车停靠点呢？",
    "query": "SELECT SUM ( B.docks_available ) AS total_available , SUM ( A.dock_count ) - SUM ( B.docks_available ) AS total_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE A.installation_date = \"8/20/2013\" AND B.time = \"2015-06-02 12:46:02\""
  },
  {
    "input": "如果建于2013年8月20日的每个车站的自行车停靠点数量都是\"Grant Avenue at Columbus Avenue\"车站自行车停靠点数量的两倍多5个，在2015年6月2日12:46:02这个时刻，那些建于2013年8月20日的车站的可用的自行车停靠点一共有多少可用，不可用的自行车停靠点呢？",
    "query": "SELECT SUM ( B.docks_available ) AS total_available , SUM ( 2 * ( SELECT dock_count FROM station WHERE name = \"Grant Avenue at Columbus Avenue\" ) + 5 - B.docks_available ) AS total_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE A.installation_date = \"8/20/2013\" AND B.time = \"2015-06-02 12:46:02\""
  },
  {
    "input": "查找持续时间是出发于“2nd at Folsom”站的行程的平均持续时间至少两倍的行程ID。",
    "query": "SELECT id FROM trip WHERE duration >= ( SELECT 2 * AVG ( duration ) FROM trip WHERE start_station_name = \"2nd at Folsom\" )"
  },
  {
    "input": "查找持续时间至少是出发于“2nd at Folsom”站的行程的平均持续时间两倍的行程ID，假如出发于“2nd at Folsom”站的行程的平均持续时间是666。",
    "query": "SELECT id FROM trip WHERE duration >= 2 * 666"
  },
  {
    "input": "2015年6月2日13:59:02，哪个车站的可用自行车数量是“2nd at Townsend”的两倍多？给出这些车站的名字。",
    "query": "SELECT B.name FROM status A JOIN station B ON A.station_id = B.id WHERE A.bikes_available > 2 * ( SELECT A.bikes_available FROM status A JOIN station B ON A.station_id = B.id WHERE B.name = \"2nd at Townsend\" AND A.time = \"2015-06-02 13:59:02\" ) AND A.time = \"2015-06-02 13:59:02\""
  },
  {
    "input": "2015年6月2日13:59:02，如果“2nd at Townsend”有5个可用的自行车，哪个车站的可用自行车是“2nd at Townsend”的两倍多？给出这些车站的名字。",
    "query": "SELECT B.name FROM status A JOIN station B ON A.station_id = B.id WHERE A.bikes_available > 2 * 5 AND A.time = \"2015-06-02 13:59:02\""
  },
  {
    "input": "对于每个车站，找出以它为终点站的行程的最小持续时间、最长持续时间以及它们的差值。",
    "query": "SELECT end_station_name , MAX ( duration ) AS max_duration , MIN ( duration ) AS min_duration , MAX ( duration ) - MIN ( duration ) AS diff FROM trip GROUP BY end_station_name"
  },
  {
    "input": "邮编为94107的地方哪天温差最大？",
    "query": "SELECT date FROM weather WHERE zip_code = \"94107\" AND max_temperature_f - min_temperature_f = ( SELECT MAX ( max_temperature_f - min_temperature_f ) FROM weather WHERE zip_code = \"94107\" )"
  },
  {
    "input": "2013年一战停战纪念日，哪个地方的平均能见度英里数最高，给出那里的邮编，以及那里那天的温差。",
    "query": "SELECT zip_code , tem_diff FROM ( SELECT zip_code , MAX ( mean_visibility_miles ) , max_temperature_f - min_temperature_f AS tem_diff FROM weather WHERE date = \"11/11/2013\" )"
  },
  {
    "input": "2013年一战停战纪念日，哪个地方的平均能见度英里数最高？给出那里的邮编。如果那里最低温度是52华氏度，那天的温差是多少？",
    "query": "SELECT zip_code , tem_diff FROM ( SELECT zip_code , MAX ( mean_visibility_miles ) , max_temperature_f - 52 AS tem_diff FROM weather WHERE date = \"11/11/2013\" )"
  },
  {
    "input": "邮编为94107的地方哪天平均温度最高？列出日期和平均摄氏温度。",
    "query": "SELECT date , 1.0 * ( MAX ( mean_temperature_f ) - 32 ) * 5 / 9 AS celsius FROM weather WHERE zip_code = \"94107\""
  },
  {
    "input": "各个地方的邮编是多少？它们2014年1月1日平均风速分别是多少千米每小时？",
    "query": "SELECT zip_code , mean_wind_speed_mph * 1.609344 AS km_per_h FROM weather WHERE date = \"1/1/2014\""
  },
  {
    "input": "邮编地址为94041的地方2014年11月12日和2014年3月12日这两天的降水量分别是多少毫米，相差多少？",
    "query": "SELECT mm_1 , mm_2 , ABS ( mm_1 - mm_2 ) AS diff FROM ( SELECT precipitation_inches * 25 AS mm_1 FROM weather WHERE zip_code = \"94041\" AND date = \"12/11/2014\" ) JOIN ( SELECT precipitation_inches * 25 AS mm_2 FROM weather WHERE zip_code = \"94041\" AND date = \"12/3/2014\" )"
  },
  {
    "input": "如果邮编地址为94041的地方2014年11月12日和2014年3月12日那两天的降水量减少了15%，那两天的降水量分别是多少毫米，相差多少？",
    "query": "SELECT mm_1 , mm_2 , ABS ( mm_1 - mm_2 ) AS diff FROM ( SELECT precipitation_inches * ( 1 - 0.15 ) * 25 AS mm_1 FROM weather WHERE zip_code = \"94041\" AND date = \"12/11/2014\" ) JOIN ( SELECT precipitation_inches * ( 1 - 0.15 ) * 25 AS mm_2 FROM weather WHERE zip_code = \"94041\" AND date = \"12/3/2014\" )"
  },
  {
    "input": "2013年国民悲痛意识日这天，各个地方的最高能见度、平均能见度、最低能见度是多少米？同时给出对应的邮编。",
    "query": "SELECT zip_code , max_visibility_miles * 1609.344 AS max_m , mean_visibility_miles * 1609.344 AS mean_m , min_visibility_miles * 1609.344 AS min_m FROM weather WHERE date = \"8/30/2013\""
  },
  {
    "input": "如果给自行车停靠点最少的车站再添10个自行车停靠点，它会有多少自行车停靠点？",
    "query": "SELECT MIN ( dock_count ) + 10 AS n_dock FROM station"
  },
  {
    "input": "政府计划在加利福尼亚州的人口排名第三的城市中再建立四个车站，那里将总共有几个车站？",
    "query": "SELECT COUNT ( * ) + 4 AS n_stations FROM station WHERE city = \"San Jose\""
  },
  {
    "input": "给出各地区的邮政编码，并计算2014年10月24日，各地区的最高温度是多少？与地球上有记录以来的最高气温相差多少？",
    "query": "SELECT zip_code , max_temperature_f , ABS ( max_temperature_f - 136 ) AS diff FROM weather WHERE date = \"10/24/2014\""
  },
  {
    "input": "旧金山有百分之多少的车站有超过25个自行车停靠点？",
    "query": "SELECT 100.0 * COUNT ( * ) / ( SELECT COUNT ( * ) FROM station WHERE city = \"San Francisco\" ) AS propotion FROM station WHERE city = \"San Francisco\" AND dock_count > 25"
  },
  {
    "input": "拥有最多车站的城市的车站数量是拥有最少车站的城市的车站数量的多少倍？",
    "query": "SELECT 1.0 * MAX ( n_station ) / MIN ( n_station ) AS times FROM ( SELECT city , COUNT ( * ) AS n_station FROM station GROUP BY city )"
  },
  {
    "input": "温差最大的时候最高温度是最低温度的多少倍？",
    "query": "SELECT 1.0 * max_temperature_f / min_temperature_f AS times FROM weather WHERE max_temperature_f - min_temperature_f = ( SELECT MAX ( max_temperature_f - min_temperature_f ) FROM weather )"
  },
  {
    "input": "邮编为94063的地方平均最高气温是多少摄氏度，比平均最低气温高多少？",
    "query": "SELECT ( AVG ( max_temperature_f ) - 32 ) * 5 / 9 AS avg_max_celsius , ( ( AVG ( max_temperature_f ) - AVG ( min_temperature_f ) ) ) * 5 / 9 AS diff_celsius FROM weather WHERE zip_code = \"94063\""
  }
]






创建few shot prompt

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate
# few shot message prompt template需要example的prompt，examples，input_variables
examples_prompt = ChatPromptTemplate.from_messages(
    [
        ('human', "{input}\n 下面是问题对应的SQL查询语句："),
        ('ai', "{query}"),
    ]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=examples_prompt,
    input_variables=["input"]
)
# 使用 Few-Shot 示例 + 新的问题，拼接出最终用于模型调用的提示内容（prompt）, 它不会生成 SQL
print(few_shot_prompt.format(input='以摄氏度为单位返回邮编为94063的地方的平均最高气温。它比当地的平均最低气温高多少？'))

Human: 2015年6月2日12:47:02，圣何塞文娱中心站不可用的自行车停靠点有多少？
 下面是问题对应的SQL查询语句：
AI: SELECT A.dock_count - B.docks_available AS n_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE B.time = "2015-06-02 12:47:02" AND A.name = "San Jose Civic Center"
Human: 如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？
 下面是问题对应的SQL查询语句：
AI: SELECT 20 - B.docks_available AS n_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE B.time = "2015-06-02 12:47:02" AND A.name = "San Jose Civic Center"
Human: 金门之城总共有多少个自行车站，比帕洛阿托少多少？
 下面是问题对应的SQL查询语句：
AI: SELECT ( SELECT COUNT ( * ) FROM station WHERE city = "Palo Alto" ) - COUNT ( * ) AS diff FROM station WHERE city = "San Francisco"
Human: 金门之城总共有多少个自行车站？如果帕洛阿托的自行车站数量是它自己的两倍还多10个，那金门之城的自行车站数量比帕洛阿托少多少？
 下面是问题对应的SQL查询语句：
AI: SELECT ( SELECT COUNT ( * ) * 2 + 10 FROM station WHERE city = "Palo Alto" ) - COUNT ( * ) AS diff FROM station WHERE city = "San Francisco"
Human: 在2015年6月2日12:46:02这个时刻，那些建于2013年8月20日的车站的可用自行车停靠点一共有多少，不可用的

## dynamic few shot example selection

动态few-shot示例选择的概念：
相比于静态的、固定的few-shot示例，动态示例选择会根据当前用户查询的具体语境，智能挑选最相关的示例给模型。这样模型得到的“指导”更有针对性，能够更准确地理解和生成符合查询需求的SQL语句。

这里就用到了rag

In [55]:
import os
os.environ["CHROMA_TELEMETRY_DISABLED"] = "1"


In [8]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
     examples,
     OpenAIEmbeddings(
         base_url="https://api.gpt.ge/v1", 
         api_key="sk-zZLix18RO29mDPcI7dB4BdB8392241D4B56e76F0CbA3Cf76"
     ),
     vectorstore,
     k=2,
     input_keys=["input"],
 )
example_selector.select_examples({"input": "邮编为94107的地方哪天平均温度最高？列出日期和平均摄氏温度。"})



C:\Users\felix\AppData\Local\Temp\ipykernel_18552\1417774857.py:5: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma()
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


[{'query': 'SELECT date , 1.0 * ( MAX ( mean_temperature_f ) - 32 ) * 5 / 9 AS celsius FROM weather WHERE zip_code = "94107"',
  'input': '邮编为94107的地方哪天平均温度最高？列出日期和平均摄氏温度。'},
 {'input': '邮编为94107的地方哪天温差最大？',
  'query': 'SELECT date FROM weather WHERE zip_code = "94107" AND max_temperature_f - min_temperature_f = ( SELECT MAX ( max_temperature_f - min_temperature_f ) FROM weather WHERE zip_code = "94107" )'}]

In [9]:
# 再在few shot prompt中添加动态选择的示例，这样就会选择最有可能的示例加入到llm prompt中
# 先检索，再加入到提示词
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_selector=example_selector, 
    example_prompt=examples_prompt,
    input_variables=["input"]
)
# 使用 Few-Shot 示例 + 新的问题，拼接出最终用于模型调用的提示内容（prompt）, 它不会生成 SQL
print(few_shot_prompt.format(input='如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？'))

Human: 如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？
 下面是问题对应的SQL查询语句：
AI: SELECT 20 - B.docks_available AS n_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE B.time = "2015-06-02 12:47:02" AND A.name = "San Jose Civic Center"
Human: 2015年6月2日12:47:02，圣何塞文娱中心站不可用的自行车停靠点有多少？
 下面是问题对应的SQL查询语句：
AI: SELECT A.dock_count - B.docks_available AS n_unavailable FROM station A JOIN status B ON A.id = B.station_id WHERE B.time = "2015-06-02 12:47:02" AND A.name = "San Jose Civic Center"


In [29]:
# 构建最后输入进模型的提示词：系统+few_shot+human input
final_prompt = ChatPromptTemplate.from_messages(
    [   
        ("system", 
"""你是一位 SQLite 数据库专家。你需要根据用户提出的问题，生成一条**语法正确、可直接执行的 SQLite 查询语句**。

请务必注意以下要求：
1. **只返回 SQL 语句本身**，不要添加任何解释、注释或 Markdown 格式（如```sql）。
2. 1 英寸这里都统一换算成25 毫米
3. 涉及到使用**日期**或者**时间**的字段做查询条件时，要仔细先查看表的日期字段的格式是什么，有的是dd/mm/yyyy，，有的是YYYY-MM-DD，要使用对应的正确的日期格式
4. 除非问题中有特别说明，默认使用标准 SQLite 语法。

下面是数据库中相关表的信息：
{table_info}

以下是一些自然语言问题及其对应的 SQL 查询示例，在你编写sql的时候可以借鉴参考（仅供参考）：
"""),

        few_shot_prompt,
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？",table_info=db.table_info,top_k=2))

System: 你是一位 SQLite 数据库专家。你需要根据用户提出的问题，生成一条**语法正确、可直接执行的 SQLite 查询语句**。

请务必注意以下要求：
1. **只返回 SQL 语句本身**，不要添加任何解释、注释或 Markdown 格式（如```sql）。
2. 1 英寸这里都统一换算成25 毫米
3. 涉及到使用**日期**或者**时间**的字段做查询条件时，要仔细先查看表的日期字段的格式是什么，有的是dd/mm/yyyy，，有的是YYYY-MM-DD，要使用对应的正确的日期格式
4. 除非问题中有特别说明，默认使用标准 SQLite 语法。

下面是数据库中相关表的信息：

CREATE TABLE station (
	id INTEGER, 
	name TEXT, 
	lat NUMERIC, 
	long NUMERIC, 
	dock_count INTEGER, 
	city TEXT, 
	installation_date TEXT, 
	PRIMARY KEY (id)
)

/*
3 rows from station table:
id	name	lat	long	dock_count	city	installation_date
2	San Jose Diridon Caltrain Station	37.3297320000	-121.9017820000	27	San Jose	8/6/2013
3	San Jose Civic Center	37.3306980000	-121.8889790000	15	San Jose	8/5/2013
4	Santa Clara at Almaden	37.3339880000	-121.8949020000	11	San Jose	8/6/2013
*/


CREATE TABLE status (
	station_id INTEGER, 
	bikes_available INTEGER, 
	docks_available INTEGER, 
	time TEXT, 
	FOREIGN KEY(station_id) REFERENCES station (id)
)

/*
3 rows from status table:
station_id	bikes_a

In [30]:
# 再次测试：sql生成
from langchain_core.runnables import RunnablePassthrough
final_prompt = final_prompt.partial(top_k=2)
generate_query = create_sql_query_chain(llm,db,final_prompt)
chain = RunnablePassthrough.assign(query=generate_query).pick("query")
#chain.invoke({"question":"如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？"})

## dynamic table selection

#### 根据表的描述，先用agent选择最有可能的表，再传入模型来生成sql

这样可以大大减少token的传入，而且更具有目标性

In [31]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd
# 遍历 CSV 的每一行，把表名和表描述拼成一个长字符串
# 格式：
#Table Name: station
#Table Description: 存储共享单车站点信息，包括站点ID、名称、经纬度...
def get_table_details():
    table_description = pd.read_csv('bike_table_description.csv')
    # 追加
    table_details = ""
    for i, row in table_description.iterrows():
        table_details = table_details + "表名："+ row['Table'] +"\n"+"表的基本概述："+row['Description']+"\n\n"

    return table_details

# 定义一个数据模板，把数据转换成结构化的、类型安全的 Python 对象
# 不用pydantic就会有时返回 markdown 表格、有时自然语言、有时逗号分隔、有时没返回想要的
class Table(BaseModel):
    name: str = Field(description='Name of table in sql database')
table_details = get_table_details()
print(table_details)

表名：station
表的基本概述：记录每个共享单车站点的**固定基础信息**，包括站点ID（id）、名称（name）、经纬度（lat, long）、车桩总数量（dock_count）、所在城市（city）和安装日期（installation_date,格式为 dd/mm/yyyy 比如：8/12/2024）。该表信息**不会随时间变化**，用于查询站点本身的属性、地理位置或容量。\n\n常见用户问题示例：\n- 哪个城市的站点最多？\n- 每个站点有多少个停放桩？\n- 安装日期最早的站点是哪个？\n- 名称包含‘Civic Center’的站点有几个？\n\n使用场景提示：当问题涉及站点总容量、位置、城市归属等基础信息时，选用此表。

表名：status
表的基本概述：记录单车站点在某一时刻的**实时状态信息**，包括站点ID、当前可用单车数量（bikes_available）、空闲车桩数量（docks_available）和状态更新时间（time）。\n\n常见用户问题示例：\n- 2015年6月2日12:47某站点还有多少空车位？\n- 哪些站点在该时间点几乎满了？\n- 所有站点某时刻的总可用单车是多少？\n\n使用场景提示：当问题涉及“某个时间点”的单车使用情况、供需状态，请选用此表。通常与 station 表联合使用以获取站名。

表名：trip
表的基本概述：记录用户的单车骑行数据，包括行程ID、骑行时长（duration）、起止时间（start_date, end_date）、起止站点、用户类型（subscription_type）及邮编信息（zip_code）等。\n\n常见用户问题示例：\n- 查找持续时间超过平均值两倍的行程ID？\n- 哪些站点是用户常用的起点或终点？\n- 哪类用户更倾向使用共享单车？\n\n使用场景提示：当问题涉及骑行行为分析、用户习惯、路线轨迹时，请选用此表。

表名：weather
表的基本概述：记录不同邮政编码区域的天气信息，包括日期（date，格式为dd/mm/yyyy）、最高/最低/平均温度、湿度、降水、风速、能见度、天气事件（events）等。\n\n常见用户问题示例：\n- 哪天的温差最大？\n- 2014年某地的降水量是多少？\n- 天气变化是否影响出行量？\n\n使用场景提示：当问题涉及气象条件、温差分

In [32]:
# 先让模型选表
table_details_prompt = f"""请返回**所有**可能与用户问题相关的 SQL 表名。
这些表包括：

{table_details}

请务必包含**所有**可能相关的表，即使你不确定它们是否真正需要。"""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
#tables = table_chain.invoke({"input": "返回2015年6月2日12点47分2秒圣何塞文娱中心站不可用的自行车停靠点的数量，假设它总共有二十个自行车停靠点。"})
#tables


In [33]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
# select_table.invoke({"question": "如果圣何塞文娱中心站站总共有20个自行车停靠点，那么2015年6月2日12:47:02，它不可用的自行车停靠点有多少？"})


## 最终版本的nl2sql agent

In [39]:
chain = (
    RunnablePassthrough.assign(table_names_to_use=select_table) |
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
)
output = chain.invoke({"question": "金门之城的自行车站总数是多少。与帕洛阿托的自行车站总数相比，它少多少？"})
print(output)
print("SQL:", output["query"])
print("Result:", output["result"])

{'question': '金门之城的自行车站总数是多少。与帕洛阿托的自行车站总数相比，它少多少？', 'table_names_to_use': ['station'], 'query': "SELECT COUNT(*) AS san_francisco_stations,  \n       (SELECT COUNT(*) FROM station WHERE city = 'Palo Alto') - COUNT(*) AS difference  \nFROM station  \nWHERE city = 'San Francisco';", 'result': '[(35, -30)]'}
SQL: SELECT COUNT(*) AS san_francisco_stations,  
       (SELECT COUNT(*) FROM station WHERE city = 'Palo Alto') - COUNT(*) AS difference  
FROM station  
WHERE city = 'San Francisco';
Result: [(35, -30)]


## 评估：准备test.json
要求格式：
| 字段                     | 说明                              | 格式示例                                   |
| ---------------------- | ------------------------------- | -------------------------------------- |
| `db_id`                | 数据库标识，表示查询针对哪个数据库               | `"bike_1"`                             |
| `question`             | 自然语言问题，模型根据它生成 SQL              | `"请问2012年后有哪些音乐会参演歌手多于Happy Tonight？"` |
| `gold_sql`             | 正确的 SQL 查询语句，用于和模型生成的 SQL 比较    | `SELECT ... FROM ... WHERE ...`        |
| `expected_result` (可选) | 执行 gold\_sql 后的正确结果，便于验证查询结果准确性 | JSON 格式或者列表形式                          |


In [19]:
import json

# 输入输出文件路径
input_path = "world_1.json"
output_path = "deduplicated.json"

# 加载原始数据
with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

# 去重逻辑
seen_queries = set()
deduplicated = []

for item in data:
    query = item.get("query", "").strip()
    if query not in seen_queries:
        deduplicated.append(item)
        seen_queries.add(query)

# 写入去重后的数据
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(deduplicated, f, ensure_ascii=False, indent=2)

print(f"去重完成，保留了 {len(deduplicated)} 条数据，已保存为 {output_path}")


去重完成，保留了 26 条数据，已保存为 deduplicated.json


In [2]:
import json

input_path = "bike_1_filtered.json"
output_path = "bike_1_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 bike_1_test_data.json ，共 54 条样本


In [2]:
import json

input_path = "concert_singer.json"
output_path = "concert_singer_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 concert_singer_test_data.json ，共 52 条样本


In [5]:
import json

input_path = "customers_and_products_contacts.json"
output_path = "customers_and_products_contacts_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 customers_and_products_contacts_test_data.json ，共 52 条样本


In [4]:
import json

input_path = "driving_school.json"
output_path = "driving_school_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 driving_school_test_data.json ，共 52 条样本


In [5]:
import json

input_path = "formula_1.json"
output_path = "formula_1_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 formula_1_test_data.json ，共 50 条样本


In [4]:
import json

input_path = "hospital_1.json"
output_path = "hospital_1_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 hospital_1_test_data.json ，共 50 条样本


In [10]:
import json

input_path = "riding_club.json"
output_path = "riding_club_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 riding_club_test_data.json ，共 52 条样本


In [4]:
import json

input_path = "soccer_1.json"
output_path = "soccer_1_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 soccer_1_test_data.json ，共 52 条样本


In [10]:
import json

input_path = "wine_1.json"
output_path = "wine_1_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 wine_1_test_data.json ，共 52 条样本


In [18]:
import json

input_path = "world_1.json"
output_path = "world_1_test_data.json"

with open(input_path, "r", encoding="utf-8") as f:
    data = json.load(f)

test_samples = []

for item in data:
    sql = item["query"]
    sql = sql.replace('\\', '')  # 去掉反斜杠
    sql = sql.replace('"', "'")  # 双引号改单引号

    sample = {
        "db_id": item["db_id"],
        "question": item["question"],
        "gold_sql": sql
    }
    test_samples.append(sample)

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(test_samples, f, ensure_ascii=False, indent=2)

print(f"转换完成，生成 {output_path} ，共 {len(test_samples)} 条样本")


转换完成，生成 world_1_test_data.json ，共 52 条样本


In [5]:
import json

# 文件路径（根据需要修改）
fewshot_path = r"D:\PythonProject\livesqlbench\livesqlbench-main\bike_1_few_shot_examples.json"
labeled_path = r"D:\PythonProject\livesqlbench\livesqlbench-main\bike_1_filtered.json"
output_path = "fewshot_augmented_1.json"

# 加载 few-shot 原始数据
with open(fewshot_path, "r", encoding="utf-8") as f:
    fewshot_data = json.load(f)

# 加载带推理类型和常识信息的数据
with open(labeled_path, "r", encoding="utf-8") as f:
    labeled_data = json.load(f)

# 建立映射：question -> 其 reasoning 和 commonsense
label_map = {
    item["question"]: {
        "reasoning_type": item["reasoning_type"],
        "commonsense_knowledge": item["commonsense_knowledge"]
    }
    for item in labeled_data
}

# 合并数据
augmented = []
for item in fewshot_data:
    match_info = label_map.get(item["input"])
    if match_info:
        item.update(match_info)
    augmented.append(item)

# 保存输出
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(augmented, f, ensure_ascii=False, indent=2)

print("✅ fewshot 数据增强完毕，已保存为：", output_path)


✅ fewshot 数据增强完毕，已保存为： fewshot_augmented_1.json


In [20]:
import json

def convert_examples(input_path, output_path):
    # 读取原始数据
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # 转换格式
    converted = []
    for item in data:
        new_item = {
            "input": item["question"],
            "query": item["query"],
            "reasoning_type": item.get("reasoning_type", ""),
            "commonsense_knowledge": item.get("commonsense_knowledge", "")
        }
        converted.append(new_item)

    # 写入新文件
    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(converted, f, indent=2, ensure_ascii=False)

    print(f"✅ 转换完成，共 {len(converted)} 条，已保存至 {output_path}")

# 示例调用
if __name__ == "__main__":
    convert_examples("deduplicated.json", "converted_examples.json")


✅ 转换完成，共 26 条，已保存至 converted_examples.json
